# Day 6

In [5]:
#
from dataclasses import dataclass

In [3]:
puzzle_input = open("./puzzle_inputs/day6.txt").read().split("\n")

In [21]:
# Race dataclass

"""  
    For every ms of holding the button, the boat adds to its speed 1mm/ms
    the distance traveled 
"""


@dataclass
class Race:
    TotalTime: int  # Total time the race last in ms
    RecordDist: int  # Record distance for the race in mm

    # PushedTime: int = 0  # Time that the competitor pressed the button
    # Distance: int = 0  # Distance covered in mm given a PushedTime value

In [22]:
# Raw data for the races
times = [
    int(t) for t in puzzle_input[0].split(":")[1].strip().split(" ") if t.isdigit()
]

record_dists = [
    int(d) for d in puzzle_input[1].split(":")[1].strip().split(" ") if d.isdigit()
]

In [23]:
race_list = [
    Race(TotalTime=time, RecordDist=dist) for time, dist in zip(times, record_dists)
]

In [24]:
race = race_list[0]
race

Race(TotalTime=42, RecordDist=284)

In [28]:
# Given a race, calculate all the possible distances that any intiger number of ms pressed gives


def Outcomes(race):
    """
    Given a race, calculate the number of ways in which the button can be held and return
    a list with the distances covered for every possible pushed time
    """
    distances = []
    for t in range(0, race.TotalTime + 1, 1):
        distances.append((race.TotalTime - t) * t)

    return distances

In [32]:
print(race, Outcomes(race))

Race(TotalTime=42, RecordDist=284) [0, 41, 80, 117, 152, 185, 216, 245, 272, 297, 320, 341, 360, 377, 392, 405, 416, 425, 432, 437, 440, 441, 440, 437, 432, 425, 416, 405, 392, 377, 360, 341, 320, 297, 272, 245, 216, 185, 152, 117, 80, 41, 0]


In [35]:
# Now we keep only the ones that beat the record distance

wining_dist = [dist for dist in Outcomes(race) if dist > race.RecordDist]
len(wining_dist)

25

In [36]:
# Lets count the number of ways to win for every race and multiply the toguether
product = 1
for race in race_list:
    outcomes = Outcomes(race)
    wining_dist = [dist for dist in Outcomes(race) if dist > race.RecordDist]
    product *= len(wining_dist)

print(product)

440000


# Part 2

The separated digits are actually one big number!

In [48]:
time = int("".join([str(t) for t in times]))
record_distance = int("".join([str(d) for d in record_dists]))
print("Time: ", time, "[ms]")
print("Record distance: ", record_distance, "[mm]")

big_race = Race(TotalTime=time, RecordDist=record_distance)

Time:  42686985 [ms]
Record distance:  284100511221341 [mm]


In [50]:
outcomes = Outcomes(big_race)
wining_dist = [dist for dist in Outcomes(big_race) if dist > big_race.RecordDist]
len(wining_dist)

26187338

---
# Day 7

In [66]:
from dataclasses import dataclass


@dataclass
class Hand:
    Cards: str  # 5 cards ordered in a string
    Bid: int
    Rank: int = 0
    Wins: int = 0
    HandType: str = "High card"
    CardEncoding: int = 0


# Card to value ordering for later
possible_cards = ["A", "K", "Q", "J", "T", "9", "8", "7", "6", "5", "4", "3", "2"]

card_to_value = {
    card: value
    for card, value in zip(
        possible_cards, [v for v in range(len(possible_cards), 0, -1)]
    )
}
print(card_to_value)

# HandType importance for later
possible_types = [
    "Five of a kind",
    "Four of a kind",
    "Full house",
    "Three of a kind",
    "Two pair",
    "One Pair",
    "High card",
]
type_to_value = {
    handtype: value
    for handtype, value in zip(
        possible_types, [v for v in range(len(possible_types), 0, -1)]
    )
}
print(type_to_value)

{'A': 13, 'K': 12, 'Q': 11, 'J': 10, 'T': 9, '9': 8, '8': 7, '7': 6, '6': 5, '5': 4, '4': 3, '3': 2, '2': 1}
{'Five of a kind': 7, 'Four of a kind': 6, 'Full house': 5, 'Three of a kind': 4, 'Two pair': 3, 'One Pair': 2, 'High card': 1}


In [68]:
# Rank the hands by the rules of poker + some modifications

# The wins of a hand is the bid times its rank

# Add the total winnings in the list

# TEST DATA

In [116]:
# Test data first:
test_input = open("./puzzle_inputs/day7test.txt").read().split("\n")

In [117]:
hand_list_test = []
for line in test_input:
    hand = Hand(Cards=line.split(" ")[0], Bid=int(line.split(" ")[1]))
    hand_list_test.append(hand)

print(hand_list_test[0])

Hand(Cards='32T3K', Bid=765, Rank=0, Wins=0, HandType='High card', CardEncoding=0)


In [118]:
# Custom functions
import pandas as pd


def encodeCards(hand: str, card_to_value, type_to_value):
    encoding = (
        card_to_value[hand.Cards[0]] * (10**8)
        + card_to_value[hand.Cards[1]] * (10**6)
        + card_to_value[hand.Cards[2]] * (10**4)
        + card_to_value[hand.Cards[3]] * (100)
        + card_to_value[hand.Cards[4]]
    )

    # Add the type as a new encoded value, from 1 to 7

    encoding = encoding + (type_to_value[hand.HandType] * (10**10))
    return encoding


def getHandType(hand: Hand):
    """
    Given a string of 5 cards, get the type of hand based on the rules of poker,
    if no condition is met, the result is not changed from the default "High card"
    """
    cards = hand.Cards

    card_count = [hand.Cards.count(c) for c in set(hand.Cards)]

    # Five of a kind
    if any([count == 5 for count in card_count]):
        return "Five of a kind"

    # Four of a kind
    if any([count == 4 for count in card_count]):
        return "Four of a kind"

    # Full house
    if any([count == 3 for count in card_count]) and any(
        [count == 2 for count in card_count]
    ):
        return "Full house"

    # Tree of a kind
    if any([count == 3 for count in card_count]):
        return "Three of a kind"

    # Two pair
    if sum([count == 2 for count in card_count]) == 2:
        return "Two pair"

    # One pair
    if any([count == 2 for count in card_count]):
        return "One Pair"

    return hand.HandType

In [119]:
for h in hand_list_test:
    h.HandType = getHandType(h)
    h.CardEncoding = encodeCards(h, card_to_value, type_to_value)
hand_list_test

[Hand(Cards='32T3K', Bid=765, Rank=0, Wins=0, HandType='One Pair', CardEncoding=20302100312),
 Hand(Cards='T55J5', Bid=684, Rank=0, Wins=0, HandType='Four of a kind', CardEncoding=61005050105),
 Hand(Cards='KK677', Bid=28, Rank=0, Wins=0, HandType='Two pair', CardEncoding=31212060707),
 Hand(Cards='KTJJT', Bid=220, Rank=0, Wins=0, HandType='Four of a kind', CardEncoding=61210010110),
 Hand(Cards='QQQJA', Bid=483, Rank=0, Wins=0, HandType='Four of a kind', CardEncoding=61111110113)]

In [120]:
# load everithing into a dataframe becouse I suck

data = [
    {
        "Cards": hand.Cards,
        "Bid": hand.Bid,
        "Rank": hand.Rank,
        "Wins": hand.Wins,
        "HandType": hand.HandType,
        "Encoding": hand.CardEncoding,
    }
    for hand in hand_list_test
]

test_df = pd.DataFrame(data)
test_df.head()

,Cards,Bid,Rank,Wins,HandType,Encoding
0,32T3K,765,0,0,One Pair,20302100312
1,T55J5,684,0,0,Four of a kind,61005050105
2,KK677,28,0,0,Two pair,31212060707
3,KTJJT,220,0,0,Four of a kind,61210010110
4,QQQJA,483,0,0,Four of a kind,61111110113


In [124]:
sorted_df = test_df.sort_values(["Encoding"], ascending=True)

sorted_df["Rank"] = [r for r in range(1, len(sorted_df) + 1, 1)]
sorted_df["Wins"] = sorted_df["Bid"] * sorted_df["Rank"]

In [125]:
sorted_df

,Cards,Bid,Rank,Wins,HandType,Encoding
0,32T3K,765,1,765,One Pair,20302100312
2,KK677,28,2,56,Two pair,31212060707
1,T55J5,684,3,2052,Four of a kind,61005050105
4,QQQJA,483,4,1932,Four of a kind,61111110113
3,KTJJT,220,5,1100,Four of a kind,61210010110


In [126]:
sum(sorted_df["Wins"])

5905

# Test passed, lets do it with the whole thing


In [108]:
import pandas as pd

puzzle_input = open("./puzzle_inputs/day7.txt").read().split("\n")

In [105]:
hand_list = []
for line in puzzle_input:
    hand = Hand(Cards=line.split(" ")[0], Bid=int(line.split(" ")[1]))
    hand_list.append(hand)

print(hand_list[0])

Hand(Cards='TQ5TT', Bid=421, Rank=0, Wins=0, HandType='High card', CardEncoding=0)


In [106]:
for h in hand_list:
    h.HandType = getHandType(h)
    h.CardEncoding = encodeCards(h, card_to_value, type_to_value)
hand_list[:4]

[Hand(Cards='TQ5TT', Bid=421, Rank=0, Wins=0, HandType='Three of a kind', CardEncoding=40911040909),
 Hand(Cards='65K2J', Bid=973, Rank=0, Wins=0, HandType='High card', CardEncoding=10504120110),
 Hand(Cards='K543T', Bid=50, Rank=0, Wins=0, HandType='High card', CardEncoding=11204030209),
 Hand(Cards='AT68Q', Bid=952, Rank=0, Wins=0, HandType='High card', CardEncoding=11309050711)]

In [112]:
# load everithing into a dataframe becouse I suck
data = [
    {
        "Cards": hand.Cards,
        "Bid": hand.Bid,
        "Rank": hand.Rank,
        "Wins": hand.Wins,
        "HandType": hand.HandType,
        "Encoding": hand.CardEncoding,
    }
    for hand in hand_list
]

hands_df = pd.DataFrame(data)
hands_df = hands_df.sort_values(["Encoding"], ascending=True)

hands_df["Rank"] = [r for r in range(1, len(hands_df) + 1, 1)]
hands_df["Wins"] = hands_df["Bid"] * hands_df["Rank"]

hands_df.head()

,Cards,Bid,Rank,Wins,HandType,Encoding
416,234T5,953,1,953,High card,10102030904
34,239TA,384,2,768,High card,10102080913
800,24798,565,3,1695,High card,10103060807
234,248T6,561,4,2244,High card,10103070905
165,257KA,932,5,4660,High card,10104061213


In [113]:
sum(hands_df["Wins"])

248812215

---
# Part 2

Now J cards are Jokers, that can act as any card! each hand must be considered as the best hand type for every possible joker in it

In [114]:
# New Card to value conversion
possible_cards = ["A", "K", "Q", "T", "9", "8", "7", "6", "5", "4", "3", "2", "J"]

card_to_value = {
    card: value
    for card, value in zip(
        possible_cards, [v for v in range(len(possible_cards), 0, -1)]
    )
}
print(card_to_value)

{'A': 13, 'K': 12, 'Q': 11, 'T': 10, '9': 9, '8': 8, '7': 7, '6': 6, '5': 5, '4': 4, '3': 3, '2': 2, 'J': 1}


In [ ]:
# Re write the logic in getHandType function to apply the special rule that J can be any card

# Repeat the encoding, with the new HandType and card_to_value

In [131]:
def getHandType2(hand):
    """
    This time 'J' cards are treated like a joker, they take the value that makes the best type
    """

    card_count = [hand.Cards.count(c) for c in set(hand.Cards) if c != "J"]

    joker_count = hand.Cards.count("J")

    # Add the joker count to the most numerous card:
    # Find the maximum count and its index

    if card_count:
        max_count = max(card_count)
        max_index = card_count.index(max_count)

        # Add the joker count to the most numerous card (only the first occurrence in case of a tie):
        card_count[max_index] += joker_count
    else:
        card_count = [joker_count]

    # Continue as in part 1:

    # Five of a kind
    if any([count == 5 for count in card_count]):
        return "Five of a kind"

    # Four of a kind
    if any([count == 4 for count in card_count]):
        return "Four of a kind"

    # Full house
    if any([count == 3 for count in card_count]) and any(
        [count == 2 for count in card_count]
    ):
        return "Full house"

    # Tree of a kind
    if any([count == 3 for count in card_count]):
        return "Three of a kind"

    # Two pair
    if sum([count == 2 for count in card_count]) == 2:
        return "Two pair"

    # One pair
    if any([count == 2 for count in card_count]):
        return "One Pair"

    return hand.HandType

In [132]:
# Re do all the steps from here and the new encoding
import pandas as pd

puzzle_input = open("./puzzle_inputs/day7.txt").read().split("\n")

In [133]:
hand_list = []
for line in puzzle_input:
    hand = Hand(Cards=line.split(" ")[0], Bid=int(line.split(" ")[1]))
    hand_list.append(hand)

print(hand_list[0])

Hand(Cards='TQ5TT', Bid=421, Rank=0, Wins=0, HandType='High card', CardEncoding=0)


In [134]:
for h in hand_list:
    h.HandType = getHandType2(h)
    h.CardEncoding = encodeCards(h, card_to_value, type_to_value)
hand_list[:4]

[Hand(Cards='TQ5TT', Bid=421, Rank=0, Wins=0, HandType='Three of a kind', CardEncoding=41011051010),
 Hand(Cards='65K2J', Bid=973, Rank=0, Wins=0, HandType='One Pair', CardEncoding=20605120201),
 Hand(Cards='K543T', Bid=50, Rank=0, Wins=0, HandType='High card', CardEncoding=11205040310),
 Hand(Cards='AT68Q', Bid=952, Rank=0, Wins=0, HandType='High card', CardEncoding=11310060811)]

In [135]:
# load everithing into a dataframe becouse I suck
data = [
    {
        "Cards": hand.Cards,
        "Bid": hand.Bid,
        "Rank": hand.Rank,
        "Wins": hand.Wins,
        "HandType": hand.HandType,
        "Encoding": hand.CardEncoding,
    }
    for hand in hand_list
]

hands_df = pd.DataFrame(data)
hands_df = hands_df.sort_values(["Encoding"], ascending=True)

hands_df["Rank"] = [r for r in range(1, len(hands_df) + 1, 1)]
hands_df["Wins"] = hands_df["Bid"] * hands_df["Rank"]

hands_df.head()

,Cards,Bid,Rank,Wins,HandType,Encoding
416,234T5,953,1,953,High card,10203041005
34,239TA,384,2,768,High card,10203091013
800,24798,565,3,1695,High card,10204070908
234,248T6,561,4,2244,High card,10204081006
165,257KA,932,5,4660,High card,10205071213


In [136]:
sum(hands_df["Wins"])

250057090

---
# Day 8

In [2]:
from dataclasses import dataclass

puzzle_input = open("./puzzle_inputs/day8.txt").read().split("\n")

instructions = puzzle_input[0]
input_list = [row for row in puzzle_input[2:]]

In [31]:
# @dataclass
# class Node:
#     Name: str
#     Left: str
#     Right: str


# # Define a list of nodes as the class shows
# node_list = []
# for node in input_list:
#     name = node.split("=")[0].strip()
#     left = node.split("=")[1].strip().split(",")[0][1:]
#     right = node.split("=")[1].strip().split(",")[1][:-1].strip()
#     node_list.append(Node(Name=name, Left=left, Right=right))

# node_list[:2], " ... ", node_list[-2:]

# TODO: repeat this using dict structure:  name : (left, right)

([Node(Name='FLG', Left='PCR', Right='CTD'),
  Node(Name='NNF', Left='CNH', Right='SPV')],
 ' ... ',
 [Node(Name='LJT', Left='RSJ', Right='JFK'),
  Node(Name='XHT', Left='KMR', Right='KMR')])

In [3]:
node_dict = {
    name: (left, right)
    for name, left, right in [
        (
            node.split("=")[0].strip(),
            node.split("=")[1].strip().split(",")[0][1:],
            node.split("=")[1].strip().split(",")[1][:-1].strip(),
        )
        for node in input_list
    ]
}
node_dict

{'FLG': ('PCR', 'CTD'),
 'NNF': ('CNH', 'SPV'),
 'LDS': ('FSN', 'SPM'),
 'NMM': ('CXD', 'LRK'),
 'MHT': ('TXF', 'KCB'),
 'RRS': ('BGH', 'HVX'),
 'HLD': ('LPJ', 'NGG'),
 'GFB': ('DSN', 'JFX'),
 'PSS': ('HDG', 'SMV'),
 'XJC': ('XMH', 'HGK'),
 'FJP': ('KJR', 'QHG'),
 'VRS': ('XKT', 'CPC'),
 'KKM': ('VGJ', 'KJF'),
 'HNR': ('KDB', 'PSK'),
 'RXS': ('NDK', 'RNH'),
 'JKB': ('RFF', 'TSX'),
 'KRN': ('BGQ', 'KGK'),
 'BCV': ('XKT', 'CPC'),
 'JFV': ('VCT', 'DJS'),
 'FXP': ('TLV', 'JDV'),
 'QJX': ('GHH', 'GSK'),
 'CDR': ('SGR', 'TQC'),
 'CTD': ('XBH', 'JJP'),
 'PVN': ('JVM', 'CSF'),
 'NKR': ('FHQ', 'SNF'),
 'NDA': ('MBM', 'SQN'),
 'KCB': ('JDG', 'CPB'),
 'SJK': ('JMT', 'JMT'),
 'HVL': ('RFQ', 'CTF'),
 'RSK': ('FXP', 'BJV'),
 'TRN': ('LVV', 'MSF'),
 'NBN': ('MPX', 'PTT'),
 'QTJ': ('BQD', 'DBR'),
 'PJC': ('MFL', 'TSD'),
 'RFJ': ('QJM', 'CVN'),
 'FCV': ('NQB', 'MJQ'),
 'NML': ('RTB', 'RTF'),
 'KGV': ('NGB', 'LDR'),
 'KRC': ('NHR', 'QCL'),
 'GSK': ('MCF', 'BRM'),
 'NQB': ('SJX', 'XBP'),
 'TFG': ('PDN', 

In [4]:
node_dict["AAA"]

('FKJ', 'QVX')

In [5]:
# Custom functions
from itertools import cycle


def getNextNodeName(name, direction: str, node_dict):
    """
    Given a node and a direction, pick the next node
    """

    if direction == "L":
        nextnode_name = node_dict[name][0]

    if direction == "R":
        nextnode_name = node_dict[name][1]

    return nextnode_name


def followPath(node_dict: dict, instructions: str, start="AAA", end="ZZZ"):
    """
    Given a list of Nodes, folow the path given by the instruction string untill the node name found is ZZZ
    if the instructions are exhausted, loop on it untill the destination node is found
    """
    steps = 0
    name = start
    for direction in cycle(instructions):
        next_name = getNextNodeName(name, direction, node_dict)
        steps += 1
        name = next_name

        if name == end:
            break

        if steps > 10**7:
            break

    return steps, name

In [6]:
steps, name = followPath(node_dict, instructions, start="AAA")
print(steps, name)

12599 ZZZ


# Part 2

In [7]:
import re

start_regex = r"..A"
end_regex = r"..Z"

starts = [name for name in node_dict.keys() if re.match(start_regex, name)]
ends = [name for name in node_dict.keys() if re.match(end_regex, name)]

ends, starts

(['BLZ', 'LXZ', 'RLZ', 'CMZ', 'PMZ', 'ZZZ'],
 ['NDA', 'AAA', 'PTA', 'PBA', 'DVA', 'HCA'])

In [8]:
def followAllPaths(node_dict, instructions, starts, ends):
    """
    Starting in all nodes that end in A, get to all nodes that end with Z simultaneously
    """

    current_nodes = starts
    steps = 0

    for direction in cycle(instructions):
        next_nodes = [
            getNextNodeName(node, direction, node_dict) for node in current_nodes
        ]
        steps += 1
        current_nodes = next_nodes

        if all([node in ends for node in current_nodes]):
            break
        if steps > 10**7:
            break

    return steps, current_nodes

In [9]:
start_regex = r"..A"
end_regex = r"..Z"

starts = [name for name in node_dict.keys() if re.match(start_regex, name)]
ends = [name for name in node_dict.keys() if re.match(end_regex, name)]

steps, current_nodes = followAllPaths(node_dict, instructions, starts=starts, ends=ends)
print(steps, current_nodes)

KeyboardInterrupt: 

In [10]:
# HINT! for every starting position, only one end position is possible!
print(followPath(node_dict, instructions, start="AAA", end="ZZZ"))
print(followPath(node_dict, instructions, start="NDA", end="RLZ"))
print(followPath(node_dict, instructions, start="PTA", end="BLZ"))

print(followPath(node_dict, instructions, start="PBA", end="PMZ"))
print(followPath(node_dict, instructions, start="DVA", end="LXZ"))
print(followPath(node_dict, instructions, start="HCA", end="CMZ"))

(12599, 'ZZZ')
(17873, 'RLZ')
(21389, 'BLZ')
(17287, 'PMZ')
(13771, 'LXZ')
(15529, 'CMZ')


In [150]:
lengths = [
    len(instructions) * i for i in range(1, 100)
]  # The paths all end when the sequence ends! ore some multiple of that length

steps = [12599, 17873, 21389, 17287, 13771, 15529]
[step in lengths for step in steps]

# Meaning that I can run the path backwards and find the steps to return to the starting position, if those also takes a multiple of the instruction's length I can make a periodic assesment and find mustiples of all frequencies

[True, True, True, True, True, True]

In [22]:
print(followPath(node_dict, instructions, start="AAA", end="ZZZ"))
print(followPath(node_dict, instructions, start="NDA", end="RLZ"))
print(followPath(node_dict, instructions, start="PTA", end="BLZ"))

print(followPath(node_dict, instructions, start="PBA", end="PMZ"))
print(followPath(node_dict, instructions, start="DVA", end="LXZ"))
print(followPath(node_dict, instructions, start="HCA", end="CMZ"))

(12599, 'ZZZ')
(17873, 'RLZ')
(21389, 'BLZ')
(17287, 'PMZ')
(13771, 'LXZ')
(15529, 'CMZ')


---
# Day 9

In [37]:
from dataclasses import dataclass


@dataclass
class Sequence:
    Values: list[int]
    Next: int = 0
    # For part 2
    Origin: int = 0

In [38]:
# Custom functions


def subSequence(seq: Sequence) -> Sequence:
    """
    for a given sequence of arbitrary length N, calculate the resulting subsequence given by the difference
    between consecutive numbers, of length N-1

    S2(i) = S1 (i+1) - S1(i)
    """

    # Make a copy of the original, avoiding the last element
    subseq = Sequence(Values=seq.Values[:-1])

    # Subtract the element subseq[i] = seq[i+1] - subseq[i] for i in range(len(seq) -1)
    for i in range(len(subseq.Values)):
        subseq.Values[i] = seq.Values[i + 1] - subseq.Values[i]

    return subseq


def generateSubsequences(seq: Sequence) -> list[Sequence]:
    """
    Generates all sunsequences from a sequence

    """
    subsequences_list = [seq]

    failsafe = 0
    while not all([val == 0 for val in seq.Values]):
        subseq = subSequence(seq)
        subsequences_list.append(subseq)

        seq = subseq
        failsafe += 1
        if failsafe > 10**5:
            break

    return subsequences_list


def generateNextValues(subseq_list: list[Sequence]):
    """
    Fills the Next attribute of all the Sequences in a list of sequences.
    Flip the sequence for the loop convinience [::-1]
    """
    for i, subseq in enumerate(subseq_list[::-1]):
        if all([val == 0 for val in subseq.Values]):
            subseq.Next = 0
        else:
            subseq.Next = subseq.Values[-1] + subseq_list[::-1][i - 1].Next

 # TEST

In [40]:
test_input = open("./puzzle_inputs/day9test.txt").read().split("\n")


@dataclass
class Sequence:
    Values: list[int]
    Next: int = 0
    # For part 2
    Origin: int = 0


sequence_list = []
for row in test_input:
    sequence_list.append(Sequence(Values=[int(digit) for digit in row.split(" ")]))

sequence_list

[Sequence(Values=[0, 3, 6, 9, 12, 15], Next=0, Origin=0),
 Sequence(Values=[1, 3, 6, 10, 15, 21], Next=0, Origin=0),
 Sequence(Values=[10, 13, 16, 21, 30, 45], Next=0, Origin=0)]

In [41]:
final_value_list = []
for sequence in sequence_list:
    subsequences_list = generateSubsequences(sequence)
    generateNextValues(subsequences_list)
    for subseq in subsequences_list:
        print(subseq)
    print(" ")
    final_value_list.append(sequence.Next)

print(final_value_list, sum(final_value_list))

Sequence(Values=[0, 3, 6, 9, 12, 15], Next=18, Origin=0)
Sequence(Values=[3, 3, 3, 3, 3], Next=3, Origin=0)
Sequence(Values=[0, 0, 0, 0], Next=0, Origin=0)
 
Sequence(Values=[1, 3, 6, 10, 15, 21], Next=28, Origin=0)
Sequence(Values=[2, 3, 4, 5, 6], Next=7, Origin=0)
Sequence(Values=[1, 1, 1, 1], Next=1, Origin=0)
Sequence(Values=[0, 0, 0], Next=0, Origin=0)
 
Sequence(Values=[10, 13, 16, 21, 30, 45], Next=68, Origin=0)
Sequence(Values=[3, 3, 5, 9, 15], Next=23, Origin=0)
Sequence(Values=[0, 2, 4, 6], Next=8, Origin=0)
Sequence(Values=[2, 2, 2], Next=2, Origin=0)
Sequence(Values=[0, 0], Next=0, Origin=0)
 
[18, 28, 68] 114


# Now the whole thing

In [42]:
puzzle_input = open("./puzzle_inputs/day9.txt").read().split("\n")

sequence_list = []
for row in puzzle_input:
    sequence_list.append(Sequence(Values=[int(digit) for digit in row.split(" ")]))

sequence_list[:3]

[Sequence(Values=[12, 18, 39, 90, 199, 424, 889, 1853, 3829, 7788, 15539, 30516, 59516, 116587, 231569, 468274, 961829, 1992697, 4128498, 8487217, 17211396], Next=0, Origin=0),
 Sequence(Values=[6, 26, 54, 89, 142, 246, 466, 909, 1734, 3162, 5486, 9081, 14414, 22054, 32682, 47101, 66246, 91194, 123174, 163577, 213966], Next=0, Origin=0),
 Sequence(Values=[18, 17, 13, 13, 40, 154, 478, 1234, 2819, 5997, 12354, 25259, 51691, 105438, 212416, 419411, 808936, 1526204, 2831565, 5207158, 9577518], Next=0, Origin=0)]

In [43]:
final_value_list = []
for sequence in sequence_list:
    subsequences_list = generateSubsequences(sequence)
    generateNextValues(subsequences_list)

    final_value_list.append(subsequences_list[0].Next)
print(final_value_list)
sum(final_value_list)

[34298953, 276086, 17764405, 27289528, 30123036, 27588317, 129744, 21931494, 14074, 11955877, -7367, 20796, 15585610, 23399495, 282822, 32956902, 4563860, 510980, 101775, 12142285, 14966208, 3227, 26860708, 18186243, 29573503, 97243, 30095032, 42210485, -1805027, 20738630, 5804097, 2287499, 10593915, 5687743, 14416, 1165409, 53446, 5790800, 790848, -27, 7790600, 3448303, 18663876, 10144, -661, 21927014, 21232289, 493240, 7184255, 26037064, 830232, 1160241, 322246, 20323194, 26142240, 12903535, 632840, 13516657, 27637, 13716329, 31975668, 8356504, 2467, 2924, 85965, 20032394, 28955529, 3798, 98, 590255, 27114845, 186222, 23331387, 214326, 17195255, -16912, 21317858, 15639351, 20196643, 27904876, 17118439, 8212108, -45, 29727368, 418798, -47, 25084187, 10586484, 23819044, 399, 2924191, 5645263, 102356, 26837201, 1910, -1377727, -5491, 21392671, 575816, 8228120, 23932344, 2951330, 23537084, 20389665, 7566, 182, 1080479, -1222913, 25583157, 353055, -14996, 19531213, 12678480, 1471642, 2004

2075724761

# Part 2
Now generate the new first value, using the same logic

In [58]:
puzzle_input = open("./puzzle_inputs/day9.txt").read().split("\n")

sequence_list = []
for row in puzzle_input:
    sequence_list.append(Sequence(Values=[int(digit) for digit in row.split(" ")]))

sequence_list[:3]

[Sequence(Values=[12, 18, 39, 90, 199, 424, 889, 1853, 3829, 7788, 15539, 30516, 59516, 116587, 231569, 468274, 961829, 1992697, 4128498, 8487217, 17211396], Next=0, Origin=0),
 Sequence(Values=[6, 26, 54, 89, 142, 246, 466, 909, 1734, 3162, 5486, 9081, 14414, 22054, 32682, 47101, 66246, 91194, 123174, 163577, 213966], Next=0, Origin=0),
 Sequence(Values=[18, 17, 13, 13, 40, 154, 478, 1234, 2819, 5997, 12354, 25259, 51691, 105438, 212416, 419411, 808936, 1526204, 2831565, 5207158, 9577518], Next=0, Origin=0)]

In [59]:
# We can reuse the the function to get the subsequences (history)


def generateOriginValue(subseq_list: list[Sequence]):
    """
    Add the Origin field to all the sequences, the same way that getNextValue does
    but with the beginning instead.
    """
    for i, subseq in enumerate(subseq_list[::-1]):
        if all([val == 0 for val in subseq.Values]):
            subseq.Origin = 0
        else:
            subseq.Origin = (
                subseq.Values[0] - subseq_list[::-1][i - 1].Origin
            )  # Now we need to subtract, since we are going backwards in the sequence.

In [60]:
origin_value_list = []
for sequence in sequence_list:
    subsequences_list = generateSubsequences(sequence)
    generateOriginValue(subsequences_list)

    origin_value_list.append(subsequences_list[0].Origin)
print(origin_value_list)
sum(origin_value_list)

[7, -3, 14, -5, -3, 3, -1, -5, 5, -4, 14, -5, -4, 1, 12, 2, 15, -3, 3, 3, -5, 15, 8, 15, 3, 3, 9, 14, 6, -4, 2, 5, 13, -5, -5, 3, 8, 15, 3, -5, -4, 12, -2, 13, 10, 10, 13, 11, 1, 12, 7, 9, 12, 11, 6, 1, 10, -4, 5, 15, -3, 12, 3, -2, 11, 12, 7, 3, 10, 6, 4, 3, 12, 2, -1, -5, 15, 11, -3, -5, 8, 7, -1, 2, 6, 8, 0, 2, -5, 14, 6, -5, 1, -1, 7, 12, -2, 7, -1, -1, 6, 10, 10, 8, 9, 6, 4, 12, 4, -1, 8, 11, 7, -4, 9, -3, 3, 4, 11, 6, 10, 15, 4, 10, 15, 14, 9, 5, 6, 14, 5, 8, 5, 8, 1, 5, 0, 3, 4, 9, 14, 1, 9, 11, -1, 2, 2, 15, -3, 1, 6, 5, -3, 15, 8, -4, 8, 12, -3, 7, -1, 12, 7, 14, 14, -3, 14, -4, 9, 7, 9, -2, 13, 13, 2, 5, 14, 8, 10, 11, 8, 6, -3, 9, 11, 14, 7, -1, 6, 15, 7, -1, 5, 2, -3, -3, 0, -4, 11, 14]


1072

--- 
# Day 10

Advent of Code[About][Events][Shop][Settings][Log Out]Andrés Terrer Gómez 14*
       y(2023)[Calendar][AoC++][Sponsors][Leaderboard][Stats]
Our sponsors help make Advent of Code possible:
Fresha - Fresha's festive code spree, under the tech tree!
--- Day 10: Pipe Maze ---
You use the hang glider to ride the hot air from Desert Island all the way up to the floating metal island. This island is surprisingly cold and there definitely aren't any thermals to glide on, so you leave your hang glider behind.

You wander around for a while, but you don't find any people or animals. However, you do occasionally find signposts labeled "Hot Springs" pointing in a seemingly consistent direction; maybe you can find someone at the hot springs and ask them where the desert-machine parts are made.

The landscape here is alien; even the flowers and trees are made of metal. As you stop to admire some metal grass, you notice something metallic scurry away in your peripheral vision and jump into a big pipe! It didn't look like any animal you've ever seen; if you want a better look, you'll need to get ahead of it.

Scanning the area, you discover that the entire field you're standing on is densely packed with pipes; it was hard to tell at first because they're the same metallic silver color as the "ground". You make a quick sketch of all of the surface pipes you can see (your puzzle input).

The pipes are arranged in a two-dimensional grid of tiles:

| is a vertical pipe connecting north and south.
- is a horizontal pipe connecting east and west.
L is a 90-degree bend connecting north and east.
J is a 90-degree bend connecting north and west.
7 is a 90-degree bend connecting south and west.
F is a 90-degree bend connecting south and east.
. is ground; there is no pipe in this tile.
S is the starting position of the animal; there is a pipe on this tile, but your sketch doesn't show what shape the pipe has.
Based on the acoustics of the animal's scurrying, you're confident the pipe that contains the animal is one large, continuous loop.

For example, here is a square loop of pipe:

.....
.F-7.
.|.|.
.L-J.
.....
If the animal had entered this loop in the northwest corner, the sketch would instead look like this:

.....
.S-7.
.|.|.
.L-J.
.....
In the above diagram, the S tile is still a 90-degree F bend: you can tell because of how the adjacent pipes connect to it.

Unfortunately, there are also many pipes that aren't connected to the loop! This sketch shows the same loop as above:

-L|F7
7S-7|
L|7||
-L-J|
L|-JF
In the above diagram, you can still figure out which pipes form the main loop: they're the ones connected to S, pipes those pipes connect to, pipes those pipes connect to, and so on. Every pipe in the main loop connects to its two neighbors (including S, which will have exactly two pipes connecting to it, and which is assumed to connect back to those two pipes).

Here is a sketch that contains a slightly more complex main loop:

..F7.
.FJ|.
SJ.L7
|F--J
LJ...
Here's the same example sketch with the extra, non-main-loop pipe tiles also shown:

7-F7-
.FJ|7
SJLL7
|F--J
LJ.LJ
If you want to get out ahead of the animal, you should find the tile in the loop that is farthest from the starting position. Because the animal is in the pipe, it doesn't make sense to measure this by direct distance. Instead, you need to find the tile that would take the longest number of steps along the loop to reach from the starting point - regardless of which way around the loop the animal went.

In the first example with the square loop:

.....
.S-7.
.|.|.
.L-J.
.....
You can count the distance each tile in the loop is from the starting point like this:

.....
.012.
.1.3.
.234.
.....
In this example, the farthest point from the start is 4 steps away.

Here's the more complex loop again:

..F7.
.FJ|.
SJ.L7
|F--J
LJ...
Here are the distances for each tile on that loop:

..45.
.236.
01.78
14567
23...
Find the single giant loop starting at S. How many steps along the loop does it take to get from the starting position to the point farthest from the starting position?



In [147]:
puzzle_input = open("./puzzle_inputs/day10.txt").read().split("\n")
puzzle_input[0:4]

['-F|..FF.F77|7-777FLF7F|7-F--F--FF7F7F.FL-|-|777F-7FL7F77---|-7-77.F-LLF7FF|-F-7.FF77F|7---F7.7FF7-J7-FF.F77F--.-|.F.F7L7--J.FLL7J7L|J7L|.F77',
 '..|--7LJ7.LJF7J-|-7F|-JL7L77|JF7--L-L|.|-77.L-LL7|7.FJ|77-FJJF..F7.7JL7L|J|LJFJFFF777.LFFL||.F7||.FF.-J-F-F|7|-L-7J7J||L|JJF7J-L|L7.LF.--JL|',
 'F-7LLJF-JF|-L|JF|FLFLJ-FJ|.FLJLJJ-F|7L|7.JJ-||.FJL77L7L77F77--JJ7|-FJ||L|J7F|--FLJLF77-F-7|L7|||L77.-LJL7.JLFJ777|||.---F.L7JFF.J-LL-J-|-7FL',
 'F|L7.|.L7F--J.FL7F-||||F7L|7|FLLJFFJ-FJ|-J|.JJ-L-7L7FL7L77JLL7JJ7FFJ-F7-7.FF7-F7.LJL|77L7LJFJ|||FJ|.|J..|..F|FFF-L-LFJLFJ--|7--7.|7FJ7||.FL.']

In [62]:
# First approach
"""  
For every point in the matrix, label it (1,2,3,4,5,...) or (0,0), (0,1), (0,2) ...

Create a graph using the symbol in each cell

    If the symbol for cell (1,1) is "|", then add the node { "(1,1)": ["(0,1)","(2,1)"] }

Once the graph is construted, finding loops would be easier

Since this approach will create directed graphs, we want to only consider bidirectional edges

# Prune the graph, for edge in graph, if edge[::-1] : keep, else prune.

Once we have that, finding the diameter of the graph from the starting position will give us the answer 
(aka distance to the farthest point)
"""

'  \nFor every point in the matrix, label it (1,2,3,4,5,...) or (0,0), (0,1), (0,2) ...\n\nCreate a graph using the symbol in each cell\n\n    If the symbol for cell (1,1) is "|", then add the node { "(1,1)": ["(0,1)","(2,1)"] }\n\nOnce the graph is construted, finding loops would be easier\n'

In [173]:
# Create all the nodes in the matrix:

shape = (len(puzzle_input[0]), len(puzzle_input))
nodes = [f"({i},{j})" for i in range(shape[0]) for j in range(shape[1])]

node_letters = [puzzle_input[i][j] for i in range(shape[0]) for j in range(shape[1])]

graph = {
    (i, j): {"Symbol": ".", "Edges": [(-1, -1)]}
    for i in range(shape[0])
    for j in range(shape[1])
}

In [174]:
# Fill the graph depending on the symbol it has
for i in range(shape[0]):
    for j in range(shape[1]):
        graph[(i, j)]["Symbol"] = puzzle_input[i][j]

In [187]:
# Add two edges depending on the symbol:


def symbol2edge(i: int, j: int, symbol: str, shape=(140, 140)) -> list[tuple]:
    """
    Add edges to the graph, according to the symbol of the node

    | is a vertical pipe connecting north and south.
    - is a horizontal pipe connecting east and west.
    L is a 90-degree bend connecting north and east.
    J is a 90-degree bend connecting north and west.
    7 is a 90-degree bend connecting south and west.
    F is a 90-degree bend connecting south and east.
    . is ground; there is no pipe in this tile.
    S is the starting position of the animal; there is a pipe on this tile, but your sketch doesn't show what shape the pipe has.

    """

    new_edges = [(-1, -1), (144, 144)]

    if symbol == "|":
        new_edges.append((i + 1, j))  # South
        new_edges.append((i - 1, j))  # North

    if symbol == "-":
        new_edges.append((i, j + 1))  # East
        new_edges.append((i, j - 1))  # West

    if symbol == "L":
        new_edges.append((i - 1, j))  # North
        new_edges.append((i, j + 1))  # East

    if symbol == "J":
        new_edges.append((i - 1, j))  # North
        new_edges.append((i, j - 1))  # West

    if symbol == "7":
        new_edges.append((i + 1, j))  # South
        new_edges.append((i, j - 1))  # West

    if symbol == "F":
        new_edges.append((i + 1, j))  # South
        new_edges.append((i, j + 1))  # East

    if symbol == "S":  # The start will add all possible paths, then 2 will get pruned.
        new_edges.append((i + 1, j))  # South
        new_edges.append((i - 1, j))  # North
        new_edges.append((i, j + 1))  # East
        new_edges.append((i, j - 1))  # West

    # Remove any edges connecting to nowhere, at the edge of the map
    new_edges = [(x, y) for x, y in new_edges if 0 <= x <= 139 and 0 <= y <= 139]

    return new_edges

In [190]:
for i in range(shape[0]):
    for j in range(shape[1]):
        graph[(i, j)]["Edges"] = symbol2edge(i, j, graph[(i, j)]["Symbol"])

In [191]:
graph

{(0, 0): {'Symbol': '-', 'Edges': [(0, 1)]},
 (0, 1): {'Symbol': 'F', 'Edges': [(1, 1), (0, 2)]},
 (0, 2): {'Symbol': '|', 'Edges': [(1, 2)]},
 (0, 3): {'Symbol': '.', 'Edges': []},
 (0, 4): {'Symbol': '.', 'Edges': []},
 (0, 5): {'Symbol': 'F', 'Edges': [(1, 5), (0, 6)]},
 (0, 6): {'Symbol': 'F', 'Edges': [(1, 6), (0, 7)]},
 (0, 7): {'Symbol': '.', 'Edges': []},
 (0, 8): {'Symbol': 'F', 'Edges': [(1, 8), (0, 9)]},
 (0, 9): {'Symbol': '7', 'Edges': [(1, 9), (0, 8)]},
 (0, 10): {'Symbol': '7', 'Edges': [(1, 10), (0, 9)]},
 (0, 11): {'Symbol': '|', 'Edges': [(1, 11)]},
 (0, 12): {'Symbol': '7', 'Edges': [(1, 12), (0, 11)]},
 (0, 13): {'Symbol': '-', 'Edges': [(0, 14), (0, 12)]},
 (0, 14): {'Symbol': '7', 'Edges': [(1, 14), (0, 13)]},
 (0, 15): {'Symbol': '7', 'Edges': [(1, 15), (0, 14)]},
 (0, 16): {'Symbol': '7', 'Edges': [(1, 16), (0, 15)]},
 (0, 17): {'Symbol': 'F', 'Edges': [(1, 17), (0, 18)]},
 (0, 18): {'Symbol': 'L', 'Edges': [(0, 19)]},
 (0, 19): {'Symbol': 'F', 'Edges': [(1, 19)

In [192]:
# We now create the reciprocal graph, only adding an edge if both nodes point towards each other


def create_recirpocal_graph(directed_graph):
    """ """
    reciprocal_graph = {}

    for node, data in directed_graph.items():
        symbol = data["Symbol"]
        edges = data["Edges"]

        for neighbor in edges:
            # Check if the edge is reciprocal
            if node in directed_graph.get(neighbor, {}).get("Edges", []):
                # Add reciprocal edge to the new graph
                reciprocal_graph.setdefault(node, {"Symbol": symbol, "Edges": []})[
                    "Edges"
                ].append(neighbor)

    return reciprocal_graph

In [193]:
reciprocal_graph = create_recirpocal_graph(graph)

In [194]:
reciprocal_graph

{(0, 2): {'Symbol': '|', 'Edges': [(1, 2)]},
 (0, 6): {'Symbol': 'F', 'Edges': [(1, 6)]},
 (0, 8): {'Symbol': 'F', 'Edges': [(0, 9)]},
 (0, 9): {'Symbol': '7', 'Edges': [(0, 8)]},
 (0, 10): {'Symbol': '7', 'Edges': [(1, 10)]},
 (0, 11): {'Symbol': '|', 'Edges': [(1, 11)]},
 (0, 13): {'Symbol': '-', 'Edges': [(0, 14)]},
 (0, 14): {'Symbol': '7', 'Edges': [(1, 14), (0, 13)]},
 (0, 16): {'Symbol': '7', 'Edges': [(1, 16)]},
 (0, 19): {'Symbol': 'F', 'Edges': [(0, 20)]},
 (0, 20): {'Symbol': '7', 'Edges': [(1, 20), (0, 19)]},
 (0, 22): {'Symbol': '|', 'Edges': [(1, 22)]},
 (0, 23): {'Symbol': '7', 'Edges': [(1, 23)]},
 (0, 25): {'Symbol': 'F', 'Edges': [(1, 25), (0, 26)]},
 (0, 26): {'Symbol': '-', 'Edges': [(0, 27), (0, 25)]},
 (0, 27): {'Symbol': '-', 'Edges': [(0, 26)]},
 (0, 28): {'Symbol': 'F', 'Edges': [(1, 28), (0, 29)]},
 (0, 29): {'Symbol': '-', 'Edges': [(0, 30), (0, 28)]},
 (0, 30): {'Symbol': '-', 'Edges': [(0, 29)]},
 (0, 32): {'Symbol': 'F', 'Edges': [(0, 33)]},
 (0, 33): {'Sy

In [195]:
# The only nodes that can be part of the loop are those with 2 edges:
def extract_loops(reciprocal_graph):
    loops = {
        node: data for node, data in reciprocal_graph.items() if len(data["Edges"]) == 2
    }
    return loops


loop_graph = extract_loops(reciprocal_graph)

In [199]:
loop_graph

{(0, 14): {'Symbol': '7', 'Edges': [(1, 14), (0, 13)]},
 (0, 20): {'Symbol': '7', 'Edges': [(1, 20), (0, 19)]},
 (0, 25): {'Symbol': 'F', 'Edges': [(1, 25), (0, 26)]},
 (0, 26): {'Symbol': '-', 'Edges': [(0, 27), (0, 25)]},
 (0, 28): {'Symbol': 'F', 'Edges': [(1, 28), (0, 29)]},
 (0, 29): {'Symbol': '-', 'Edges': [(0, 30), (0, 28)]},
 (0, 34): {'Symbol': 'F', 'Edges': [(1, 34), (0, 35)]},
 (0, 47): {'Symbol': 'F', 'Edges': [(1, 47), (0, 48)]},
 (0, 48): {'Symbol': '-', 'Edges': [(0, 49), (0, 47)]},
 (0, 49): {'Symbol': '7', 'Edges': [(1, 49), (0, 48)]},
 (0, 53): {'Symbol': 'F', 'Edges': [(1, 53), (0, 54)]},
 (0, 54): {'Symbol': '7', 'Edges': [(1, 54), (0, 53)]},
 (0, 57): {'Symbol': '-', 'Edges': [(0, 58), (0, 56)]},
 (0, 71): {'Symbol': '7', 'Edges': [(1, 71), (0, 70)]},
 (0, 76): {'Symbol': 'F', 'Edges': [(1, 76), (0, 77)]},
 (0, 77): {'Symbol': '-', 'Edges': [(0, 78), (0, 76)]},
 (0, 78): {'Symbol': '7', 'Edges': [(1, 78), (0, 77)]},
 (0, 88): {'Symbol': '-', 'Edges': [(0, 89), (0,

In [207]:
# Find node S

start = [
    (i, j) for i in range(140) for j in range(140) if graph[(i, j)]["Symbol"] == "S"
][0]
start

(118, 63)

In [223]:
# Folow the loop graph and count the number of steps that takes to get back to S

node = start
next_node = loop_graph[node]["Edges"][0]  # Start in one of the options
(node, loop_graph[node]), (next_node, loop_graph[next_node])

# Keep going though the path that is not the last node I visited

[n != node for n in loop_graph[next_node]["Edges"]]

# keep track of the number of steps that it takes to get back to S

""" 
    While loop_graph["Symbol"] != "S": 
        ...
"""

[True, False]

In [244]:
# Find node S
start = [
    (i, j)
    for i in range(140)
    for j in range(140)
    if graph.get((i, j), {}).get("Symbol") == "S"
][0]

# Follow the loop graph and count the number of steps that take to get back to S
current_node = start
visited_nodes = set([])
steps = 0

while loop_graph[current_node]["Edges"]:
    next_node_options = [
        option
        for option in loop_graph[current_node]["Edges"]
        if option not in visited_nodes
    ]

    if not next_node_options:
        # No valid next nodes, break the loop
        print("No valid loop at ", current_node)
        break

    next_node = next_node_options[0]
    visited_nodes.add(next_node)

    current_node = next_node
    steps += 1

    if current_node == start:
        print("Loop traversed")
        # Reached back to the starting node, exit the loop
        break

print(f"Number of steps: {steps}")
print(f"The farthest node is : {steps//2} untis away")

Loop traversed
Number of steps: 13476
The farthest node is : 6738 untis away


# Part 2
--- Part Two ---
You quickly reach the farthest point of the loop, but the animal never emerges. Maybe its nest is within the area enclosed by the loop?

To determine whether it's even worth taking the time to search for such a nest, you should calculate how many tiles are contained within the loop. For example:

...........
.S-------7.
.|F-----7|.
.||.....||.
.||.....||.
.|L-7.F-J|.
.|..|.|..|.
.L--J.L--J.
...........
The above loop encloses merely four tiles - the two pairs of . in the southwest and southeast (marked I below). The middle . tiles (marked O below) are not in the loop. Here is the same loop again with those regions marked:

...........
.S-------7.
.|F-----7|.
.||OOOOO||.
.||OOOOO||.
.|L-7OF-J|.
.|II|O|II|.
.L--JOL--J.
.....O.....
In fact, there doesn't even need to be a full tile path to the outside for tiles to count as outside the loop - squeezing between pipes is also allowed! Here, I is still within the loop and O is still outside the loop:

..........
.S------7.
.|F----7|.
.||OOOO||.
.||OOOO||.
.|L-7F-J|.
.|II||II|.
.L--JL--J.
..........
In both of the above examples, 4 tiles are enclosed by the loop.

Here's a larger example:

.F----7F7F7F7F-7....
.|F--7||||||||FJ....
.||.FJ||||||||L7....
FJL7L7LJLJ||LJ.L-7..
L--J.L7...LJS7F-7L7.
....F-J..F7FJ|L7L7L7
....L7.F7||L7|.L7L7|
.....|FJLJ|FJ|F7|.LJ
....FJL-7.||.||||...
....L---J.LJ.LJLJ...
The above sketch has many random bits of ground, some of which are in the loop (I) and some of which are outside it (O):

OF----7F7F7F7F-7OOOO
O|F--7||||||||FJOOOO
O||OFJ||||||||L7OOOO
FJL7L7LJLJ||LJIL-7OO
L--JOL7IIILJS7F-7L7O
OOOOF-JIIF7FJ|L7L7L7
OOOOL7IF7||L7|IL7L7|
OOOOO|FJLJ|FJ|F7|OLJ
OOOOFJL-7O||O||||OOO
OOOOL---JOLJOLJLJOOO
In this larger example, 8 tiles are enclosed by the loop.

Any tile that isn't part of the main loop can count as being enclosed by the loop. Here's another example with many bits of junk pipe lying around that aren't connected to the main loop at all:

FF7FSF7F7F7F7F7F---7
L|LJ||||||||||||F--J
FL-7LJLJ||||||LJL-77
F--JF--7||LJLJ7F7FJ-
L---JF-JLJ.||-FJLJJ7
|F|F-JF---7F7-L7L|7|
|FFJF7L7F-JF7|JL---7
7-L-JL7||F7|L7F-7F7|
L.L7LFJ|||||FJL7||LJ
L7JLJL-JLJLJL--JLJ.L
Here are just the tiles that are enclosed by the loop marked with I:

FF7FSF7F7F7F7F7F---7
L|LJ||||||||||||F--J
FL-7LJLJ||||||LJL-77
F--JF--7||LJLJIF7FJ-
L---JF-JLJIIIIFJLJJ7
|F|F-JF---7IIIL7L|7|
|FFJF7L7F-JF7IIL---7
7-L-JL7||F7|L7F-7F7|
L.L7LFJ|||||FJL7||LJ
L7JLJL-JLJLJL--JLJ.L
In this last example, 10 tiles are enclosed by the loop.

Figure out whether you have time to search for the nest by calculating the area within the loop. How many tiles are enclosed by the loop?


In [ ]:
# First approach:
""" 
Assuming only one loop is present, all other combination of (i,j) are either in I or out O of the loop.

Lets re-create the map but change the non-connected pipes to "."
"""

In [259]:
new_map = [
    puzzle_input[i][j]
    for i in range(140)
    for j in range(140)
    if (i, j) in loop_graph.keys()
]

In [262]:
# Second approach:

# Find all pairs of nodes that are not in the loop
"""  
    Construct two more graphs: Inside and Outside.

    Starting in any node not in the loop, connect other nodes that are also not in the loop, keep doing it untill all nodes are either in the loop or in the
    new graphs. One of them is out the other is in.

    # BAD! 

    The puzzle says that you can squeze between two pipes ... so this wont solve it
"""

'  \n    Construct two more graphs: Inside and Outside.\n\n    Starting in any node not in the loop, connect other nodes that are also not in the loop, keep doing it untill all nodes are either in the loop or in the\n    new graphs. One of them is out the other is in.\n\n    # BAD! \n\n    The puzzle says that you can squeze between two pipes ... so this wont solve it\n'

In [261]:
# Third approach:
"""  
Create the anti-loop graph

Instead of joining nodes based on if the pipe connects them, start with a fully connected 2D grid and prune the edges that the pipes connect
"""

'  \nCreate the anti-loop graph\n\nInstead of joining nodes based on if the pipe connects them, start with a fully connected 2D grid and prune the edges that the pipes connect\n'

In [263]:
# Forth approach:

"""  
    The loop acts as a frontere (?) bwteen inside and outside, to get from any I node to another I i can use the nodes in the loop
"""

'  \n    The loop acts as a frontere (?) bwteen inside and outside, to get from any I node to another I i can use the nodes in the loop\n'

In [264]:
# Fith approach:

""" 
Consider the expanded map: a map where each node is now a 3x3 grid, extrend the paths using "|", "-" or ".". then re-calculate the ins and outs
with a simpler approach:
    ie: connect all non loop nodes with one another and count the number of nodes in each connected graph (I and O)

Keep track of which nodes are original nodes, so they can be counted later.
"""

' \nConsider the expanded map: a map where each node is now a 3x3 grid, extrend the paths using "|", "-" or ".". then re-calculate the ins and outs\nwith a simpler approach:\n    ie: connect all non loop nodes with one another and count the number of nodes in each connected graph (I and O)\n\nKeep track of which nodes are original nodes, so they can be counted later.\n'